<a href="https://colab.research.google.com/github/davidfague/Model_Reduction_Methods/blob/main/New_NMDASimulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulation for generating data for dendritic spike analysis
##Recorded Currents:

Na,K,Ca,ih,...

## synapse distribution:

10,000 random. will update to include more realistic algorithm

#### Download modules from Github

In [1]:
!pip install neuron

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 30.9 MB/s eta 0:00:00


In [2]:
!pip install neuron_reduce

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!git clone https://github.com/davidfague/Model_Reduction_Methods.git

Cloning into 'Model_Reduction_Methods'...
remote: Enumerating objects: 932, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 932 (delta 92), reused 0 (delta 0), pack-reused 774
Receiving objects: 100% (932/932), 6.00 MiB | 8.18 MiB/s, done.
Resolving deltas: 100% (521/521), done.


In [4]:
%cd Model_Reduction_Methods/

#import reduction and expansion functions
from test_neuron_reduce.subtree_reductor_func import subtree_reductor
from cable_expander_func import cable_expander

#import recording functions
from stylized_module.recorder import Recorder

#import analysis functions
from utils import make_seg_df,generate_stylized_geometry,make_reduced_seg_df,plot_morphology,check_connectivity,generate_reduced_cell_seg_coords, create_seg_var_report,plot_seg_heatmap


# from modeling_module.synapses import Synapse, Listed_Synapse
from modeling_module.cell_model import cell_model

import pandas as pd

/content/Model_Reduction_Methods


In [5]:
from neuron import h
from typing import Optional, Union, List
from modeling_module.synapses import CurrentInjection, Synapse, Listed_Synapse
import numpy as np
import pandas as pd
import math
import os
import h5py
import csv
class cell_model():
  '''expanded cell model class for ECP calculation
  takes hoc cell model and does bookkeeping for analysis functions
  '''
  def __init__(self,model,synapses_list=None,netcons_list=None,gen_3d=True,gen_geom_csv=False,spike_threshold: Optional[float] = None):
    self.all=model.all
    self.soma=model.soma
    self.apic=model.apic
    self.dend=model.dend
    self.axon=model.axon
    #convert nrn section lists to python lists if applicable
    self.all=self.__convert_sectionlist(sectionlist=self.all)
    self.soma=self.__convert_sectionlist(sectionlist=self.soma, return_singles=True) #if sectionlist contains only one section returns just the section instead of list of sections
    self.dend=self.__convert_sectionlist(sectionlist=self.dend)
    self.apic=self.__convert_sectionlist(sectionlist=self.apic)
    self.axon=self.__convert_sectionlist(sectionlist=self.axon)
    self.spike_threshold = spike_threshold
    self.synapses_list=synapses_list #list of synapse objects from model reduction
    self.netcons_list=netcons_list # list of netcon objects from model reduction
    self.segments=[] # list for nrn segment objects
    self.injection=[] # list of injection objects
    self.synapse=[] # list of python synapse class objects
    self.sec_id_lookup = {}  # dictionary from section type id to section index
    self.sec_id_in_seg = []  # index of the first segment of each section in the segment list
    self.sec_angs = [] # list of angles that were used to branch the cell
    self.sec_rots = []
    if gen_3d==True:
      self.__generate_sec_coords()
    self.__store_segments()
    self.__set_spike_recorder()
    self.__calc_seg_coords()
    self.__store_synapses_list() #store and record synapses from the synapses_list used to initialize the cell
    self.grp_ids = []
    if gen_geom_csv==True:
      self.__generate_geometry_file()
    # self.calculate_netcons_per_seg()
    self.__insert_unused_channels()
    self.__setup_recorders()

  def __calc_seg_coords(self):
      """Calculate segment coordinates for ECP calculation"""
      p0 = np.empty((self._nseg, 3))
      p1 = np.empty((self._nseg, 3))
      p05 = np.empty((self._nseg, 3))
      r = np.empty(self._nseg)
      for isec, sec in enumerate(self.all):
          iseg = self.sec_id_in_seg[isec]
          nseg = sec.nseg
          seg_length = sec.L / nseg
          for i in range(sec.n3d()):
              pt1 = None  # initialize pt1
              if i == 0:
                  pt0 = np.array([sec.x3d(i), sec.y3d(i), sec.z3d(i)])
              elif i == sec.n3d() - 1:
                  pt1 = np.array([sec.x3d(i), sec.y3d(i), sec.z3d(i)])
              else:
                  pt0 = np.array([sec.x3d(i - 1), sec.y3d(i - 1), sec.z3d(i - 1)])
                  pt1 = np.array([sec.x3d(i), sec.y3d(i), sec.z3d(i)])
              if pt1 is not None:  # only set pt1 if it was initialized
                  for j in range(nseg):
                      p0[iseg+j, :] = (pt0 + (pt1-pt0)*j/nseg + (pt1-pt0)/(2*nseg))
                      p1[iseg+j, :] = (pt0 + (pt1-pt0)*(j+1)/nseg + (pt1-pt0)/(2*nseg))
                      p05[iseg+j, :] = (pt0 + (pt1-pt0)*(j+0.5)/nseg)
                      r[iseg+j] = sec.diam / 2
          iseg += nseg
          
      self.seg_coords = {'dl': p1 - p0, 'pc': p05, 'r': r}

  def __store_segments(self):
    self.segments = []
    self.sec_id_in_seg = []
    nseg = 0
    for sec in self.all:
        self.sec_id_in_seg.append(nseg)
        nseg += sec.nseg
        for seg in sec:
            self.segments.append(seg)
            self.__store_point_processes(seg)
    self._nseg = nseg

  def __store_synapses_list(self):
    '''
    store and record synapses from the list from model reduction algorithm
    '''
    temp_list=[] # generate temp list that has each netcon's synapse obj
    for netcon in self.netcons_list:
      syn=netcon.syn()
      if syn in self.synapses_list:
        syn_seg_id=self.segments.index(netcon.syn().get_segment())
        if syn in self.segments[syn_seg_id].point_processes():
          temp_list.append(syn)
        else:
          temp_list.append(None)
          print("Warning: synapse not in designated segment's point processes")

      else:
        temp_list.append(None)
        print("Warning: potentially deleted synapse:","|NetCon obj:",netcon,"|Synapse obj:",syn,"the NetCon's synapse is not in synapses_list. Check corresponding original cell's NetCon for location, etc.")
    # now use temp list to assign each synapse its netcons
    for synapse in self.synapses_list:
      synapse_netcons=[]
      if synapse in temp_list:
        num_netcons=temp_list.count(synapse)
        START=0
        for i in range(num_netcons):
          netcon_id=temp_list.index(synapse,START) #get all the netcon indices that are pointed toward this synapse # use np.where() instead of index() to return multiple indices.
          START=netcon_id+1
          synapse_netcons.append(self.netcons_list[netcon_id])
        self.synapse.append(Listed_Synapse(synapse,synapse_netcons)) #record synapse and add to the list
      else:
        print('Warning: ', synapse, 'does not have any netcons pointing at it. if synapse is None then deleted synapse may be stored in synapses_list')

  def __store_point_processes(self,seg):
    for pp in seg.point_processes():
        self.injection.append(pp)

  def __set_spike_recorder(self, threshold: Optional = None):
      if threshold is not None:
          self.spike_threshold = threshold
      if self.spike_threshold is None:
          self.spikes = None
      else:
          vec = h.Vector()
          nc = h.NetCon(self.soma(0.5)._ref_v, None, sec=self.soma)
          nc.threshold = self.spike_threshold
          nc.record(vec)
          self.spikes = vec

  def add_injection(self, sec_index, **kwargs):
        """Add current injection to a section by its index"""
        self.injection.append(CurrentInjection(self, sec_index, **kwargs))

  def add_synapse(self, stim: h.NetStim, sec_index: int, **kwargs):
        """Add synapse to a section by its index"""
        new_syn=Synapse(self, stim, sec_index, **kwargs)
        self.netcons_list.append(new_syn.nc)
        self.synapse.append(new_syn)

  def __generate_sec_coords(self):
      '''
      Note: need to improve branching so that it is random direction in a quadrant of a sphere rather than x-y plane
      takes a cell that has no n3d() coordinates and gives new coordinates
      by choosing an arbitrary direction for the subtree to move
      '''
      section_obj_list= self.all
      # print(section_obj_list)
      axial=False
      parent_sections=[] #list for already seen parent_sections of this type
      for sec in section_obj_list:
        sec_length=sec.L
        if sec is self.soma:
          self.sec_angs.append(0)
          self.sec_rots.append(0)
          # pt0 = [0., -1 * sec.diam, 0.] #does not seem to preserve soma shape , but need to make sure soma children begin at correct 3d coordinate.
          # pt1 = [0., 0., 0.]
          # sec.pt3dclear()
          # sec.pt3dadd(*pt0, sec.diam)
          # sec.pt3dadd(*pt1, sec.diam)
          if sec.nseg != 1:
            print('Changing soma nseg from',sec.nseg,'to 1')
            sec.nseg = 1
        else:
          if sec.parentseg() is not None:
            psec=sec.parentseg().sec
            if (psec in self.apic) and (psec is not self.apic[0]): # branch
              # print('branch')
              nbranch = len(psec.children())
            else:
              nbranch=1
          else:
            psec=None # may need to provide more implementation in the case of no 3d coords and no parent section.
            nbranch=1

          # rot = 2 * math.pi/nbranch
          rot=np.random.uniform(low=0,high=2*np.pi)# rot can be used to uniformly rotate branches if i=parent_sections.count(psec) and rot = 2 * math.pi/nbranch

          i=1 #i can be used to uniformly rotate the sections if rot = 2 * math.pi/nbranch and i=parent_sections.count(psec)
          # if nbranch==1:
          #   i=1
          # else:
          #   i=parent_sections.count(psec)

          parent_sections.append(psec)
          # print("sec: ",sec, "|nbranch: ",nbranch,"|i: ,",i,"|parent_sections:",parent_sections)
          length=sec.L
          diameter=sec.diam
          fullsecname = sec.name()
          # print(fullsecname)
          sec_type = fullsecname.split(".")[1][:4]
          # print(sec_type)
          if sec_type == "apic":
            if sec==self.apic[0]: # trunk
              ang=1.570796327
            else:
              # ang=np.random.uniform(low=0,high=np.pi) #branches
              ang=np.random.normal(loc=np.pi/2,scale=0.5) # could add limits to ang (if ang>val:ang=val)
          elif sec_type=="dend":
            # ang=-np.random.uniform(low=0,high=np.pi)
            ang=-np.random.normal(loc=np.pi/2,scale=0.5) # could add limits to ang (if ang>val:ang=val)
          elif sec_type=="axon":
            ang=-1.570796327
          else:
            print(sec,sec_type,' is not apic, dend or axon')
            ang=0
          if axial == True:
            x = 0
            y = length*((ang>=0)*2-1)
          else:
            x = length * math.cos(ang)
            y = length * math.sin(ang)
          self.sec_angs.append(ang)
          self.sec_rots.append(i*rot)
          #find starting position #need to update to use parent segment coordinates instead of using first section coordinate
          pt0 = [psec.x3d(1), psec.y3d(1), psec.z3d(1)]
          pt1 = [0., 0., 0.]
          pt1[1] = pt0[1] + y
          pt1[0] = pt0[0] + x * math.cos(i * rot)
          pt1[2] = pt0[2] + x * math.sin(i * rot)
          # print(sec,i*rot)
          sec.pt3dclear()
          sec.pt3dadd(*pt0, sec.diam)
          sec.pt3dadd(*pt1, sec.diam)
        if int(sec.L) != int(sec_length):
          print('Error: generation of 3D coordinates resulted in change of section length for',sec,'from',sec_length,'to',sec.L)

  def __generate_geometry_file(self):
    '''
    generates geometry file specifying name, pid, ang, radius, length, type
    work in progress
    '''
    df = pd.DataFrame()
    ids=[]
    names=[]
    types=[]
    pids=[]
    axials=[]
    nbranchs=[]
    Ls=[]
    Rs=[]
    angs=self.sec_angs
    rots=self.sec_rots
    for sec in self.all:
      # print(dir(sec))
      name=sec.name()
      # print(name)
      names.append(name)
      ids.append(names.index(name))
      _,sec_type_withinteger=name.split('.')
      sec_type,_=sec_type_withinteger.split('[')
      types.append(sec_type)
      pseg = sec.parentseg()
      if pseg == None:
        pids.append(None)
      else:
        psec=pseg.sec
        px3d=psec.x3d
        pids.append(int(names.index(psec.name())))
      # axials.append('TRUE')
      # nbranchs.append(1)
      Ls.append(sec.L)
      # print(dir(sec))
      Rs.append(sec.diam/2)
    df['id']=ids
    df['name']=names
    df['pid']=pids
    df['type']=types
    df['L']=Ls
    df['R']=Rs
    try:df['ang']=angs
    except:pass
    df['rot']=rots
    # df['axials']=axials # may need to fix
    # df['nbranch']=nbranchs # may need to fix
    self.geometry=df

  def __convert_sectionlist(self,sectionlist,return_singles=False):
    '''
    convert nrn sectionlist objects to python list
    return_singles set to true will return section instead of [section] for lists with only one section
    '''
    new_sectionlist=[]
    if str(type(sectionlist)) == "<class 'hoc.HocObject'>":
      for sec in sectionlist:
        new_sectionlist.append(sec)
    else:
      new_sectionlist=sectionlist
    if return_singles==True:
      if str(type(new_sectionlist))!="<class 'nrn.Section'>":
        if len(new_sectionlist)==1:
          new_sectionlist=new_sectionlist[0]
    return new_sectionlist

  def __insert_unused_channels(self):
      channels = [('NaTa_t', 'gNaTa_t_NaTa_t', 'gNaTa_tbar'),
                  ('Ca_LVAst', 'ica_Ca_LVAst', 'gCa_LVAstbar'),
                  ('Ca_HVA', 'ica_Ca_HVA', 'gCa_HVAbar'),
                  ('Ih', 'ihcn_Ih', 'gIhbar')]
      for channel, attr, conductance in channels:
          for sec in self.all:
              if not hasattr(sec(0.5), attr):
                  sec.insert(channel)
                  for seg in sec:
                      setattr(getattr(seg, channel), conductance, 0)
                  # print(channel, sec) # empty sections

  def __setup_recorders(self):
      self.gNaTa_T = Recorder(obj_list=self.segments, var_name='gNaTa_t_NaTa_t')
      self.ina = Recorder(obj_list=self.segments, var_name='ina_NaTa_t')
      self.ical = Recorder(obj_list=self.segments, var_name='ica_Ca_LVAst')
      self.icah = Recorder(obj_list=self.segments, var_name='ica_Ca_HVA')
      self.ih = Recorder(obj_list=self.segments, var_name='ihcn_Ih')
      self.Vm = Recorder(obj_list=self.segments)

  def __create_output_folder(self):
      nbranches = len(self.apic)-1
      nc_count = len(self.netcons_list)
      syn_count = len(self.synapses_list)
      seg_count = len(self.segments)
      

      self.output_folder_name = (
          str(h.tstop)+
          "outputcontrol_" +
          str(nbranches) + "nbranch_" +
          str(nc_count) + "NCs_" +
          str(syn_count) + "nsyn_" +
          str(seg_count) + "nseg"
      )

      if not os.path.exists(self.output_folder_name):
          print('Outputting data to ', self.output_folder_name)
          os.makedirs(self.output_folder_name)

      return self.output_folder_name

  def get_recorder_data(self):
      '''
      Method for calculating net synaptic currents and getting data after simulation
      '''
      numTstep = int(h.tstop/h.dt)
      i_NMDA_bySeg = [[0] * (numTstep+1)] * len(self.segments)
      i_AMPA_bySeg = [[0] * (numTstep+1)] * len(self.segments)
      # i_bySeg = [[0] * (numTstep+1)] * len(self.segments)

      for synapse in self.synapses_list:
          try:
              i_NMDA = np.array(synapse.rec_vec.vec_list[1])
              i_AMPA = np.array(synapse.rec_vec.vec_list[0])
              seg = synapse.get_segment_id()

              try:
                  i_NMDA_bySeg[seg] = i_NMDA_bySeg[seg] + i_NMDA
                  i_AMPA_bySeg[seg] = i_AMPA_bySeg[seg] + i_AMPA
              except:
                  pass
          except:
              continue

      i_NMDA_df = pd.DataFrame(i_NMDA_bySeg) * 1000
      i_AMPA_df = pd.DataFrame(i_AMPA_bySeg) * 1000
      

      self.data_dict = {}
      self.data_dict['spikes']=self.spikes
      self.data_dict['ih_data'] = self.ih.as_numpy()
      self.data_dict['gNaTa_T_data'] = self.gNaTa_T.as_numpy()
      self.data_dict['ina_data'] = self.ina.as_numpy()
      self.data_dict['icah_data'] = self.icah.as_numpy()
      self.data_dict['ical_data'] = self.ical.as_numpy()
      self.data_dict['Vm'] = self.Vm.as_numpy()
      self.data_dict['i_NMDA'] = i_NMDA_df
      self.data_dict['i_AMPA'] = i_AMPA_df
      # self.data_dict['i'] = i_bySeg
      self.__create_output_files(self.__create_output_folder())

      return self.data_dict

  def __create_output_files(self,output_folder_name):
      for name, data in self.data_dict.items():
        try:
          self.__report_data(f"{output_folder_name}/{name}_report.h5", data.T)
        except:
          self.__report_data(f"{output_folder_name}/{name}_report.h5", data)

  def __report_data(self,reportname, dataname):
      try:
          os.remove(reportname)
      except FileNotFoundError:
          pass

      with h5py.File(reportname, 'w') as f:
          f.create_dataset("report/biophysical/data", data=dataname)

  def plot_seg_heatmap(self, seg_df, color_column):
      '''
      Plots a heatmap of a segment dataframe, using a specified column for color
      color_column  :   attribute that is per segment
      Can update segments dataframe so that is instead a segment class with the option of saving the dataframe when initializeing the cell?
      '''
      if isinstance(getattr(self, color_column), list):
          color_data = np.concatenate(getattr(self, color_column))
      else:
          color_data = getattr(self, color_column)

      label = color_column.capitalize()
      savename = color_column.lower()

      plt.figure(figsize=(4,10))
      ax = plt.scatter(seg_df["Coord X"], seg_df["Coord Y"],c = color_data, cmap='jet',)
      plt.vlines(110,400,500)
      plt.text(0,450,'100 um')
      plt.hlines(400,110,210)
      plt.text(110,350,'100 um')
      plt.xticks([])
      plt.yticks([])
      cbar = plt.colorbar()
      cbar.ax.set_ylabel(label, rotation=270)

      plt.box(False)
      plt.savefig("/"+self.output_folder_name+savename+'.svg')

  def write_seg_info_to_csv(self):
      seg_info=self.__get_segment_info__()
      with open(self.output_folder_name+'/seg_info.csv', mode='w') as file:
          writer = csv.DictWriter(file, fieldnames=seg_info[0].keys())
          writer.writeheader()
          for row in seg_info:
              writer.writerow(row)

  def __get_segment_info__(self):      
      seg_info = []
      k = 0
      j = 0
      for sec in self.all:
          sec_type = sec.name().split('.')[1][:4]
          for i, seg in enumerate(sec):
              seg_info.append({
                  'seg': seg,
                  'seg_id': j,
                  'bmtk_id': k,
                  'x': seg.x,
                  'Type': sec_type,
                  'Sec ID': int(sec.name().split('[')[2].split(']')[0]),
                  'nseg': seg.sec.nseg,
                  'Ra': seg.sec.Ra,
                  'seg_L': sec.L/sec.nseg,
              })
              j += 1
          k += 1
      return self.__get_parent_segment_ids(seg_info)

  def __get_parent_segment_ids(self, seg_info):
      for seg in seg_info:
          seg['parent_seg_id'] = None
      pseg_ids = []
      for i, seg in enumerate(seg_info):
          idx = int(np.floor(seg['x'] * seg['nseg']))
          if idx != 0:
              pseg_id = i-1
          else:
              pseg = seg['seg'].sec.parentseg()
              if pseg is None:
                  pseg_id = None
              else:
                  psec = pseg.sec
                  nseg = psec.nseg
                  pidx = int(np.floor(pseg.x * nseg))
                  if pseg.x == 1.:
                      pidx -= 1
                  try:
                      pseg_id = next(idx for idx, info in enumerate(seg_info) if info['seg'] == psec((pidx + .5) / nseg))
                  except StopIteration:
                      pseg_id = "Segment not in segments"
              seg_info[i]['parent_seg_id'] = pseg_id
          # pseg_ids.append(pseg_id)
      return self.__get_segment_elec_dist(seg_info)

  def __get_segment_elec_dist(self, seg_info):
      for seg in seg_info:
          seg['seg_elec_info'] = {}
      freqs = {'delta': 1, 'theta': 4, 'alpha': 8, 'beta': 12, 'gamma': 30}

      soma_passive_imp = h.Impedance()
      soma_active_imp = h.Impedance()
      nexus_passive_imp = h.Impedance()
      nexus_active_imp = h.Impedance()
      try:
          soma_passive_imp.loc(self.hobj.soma[0](0.5))
          soma_active_imp.loc(self.hobj.soma[0](0.5))
      except:
          try:
              soma_passive_imp.loc(self.soma[0](0.5))
              soma_active_imp.loc(self.soma[0](0.5))
          except:
              try:
                  soma_passive_imp.loc(self.soma(0.5))
                  soma_active_imp.loc(self.soma(0.5))
              except:
                  raise AttributeError("Could not locate soma for impedance calculation")
      try:
          nexus_passive_imp.loc(self.hobj.apic[0](0.99))
          nexus_active_imp.loc(self.hobj.apic[0](0.99))
      except:
          try:
              nexus_passive_imp.loc(self.apic[0](0.99))
              nexus_active_imp.loc(self.apic[0](0.99))
          except:
              try:
                  nexus_passive_imp.loc(self.apic(0.99))
                  nexus_active_imp.loc(self.apic(0.99))
              except:
                  raise AttributeError("Could not locate the nexus for impedance calculation")

      for freq_name, freq_hz in freqs.items():
          soma_passive_imp.compute(freq_hz + 1 / 9e9, 0) #passive from soma
          soma_active_imp.compute(freq_hz + 1 / 9e9, 1) #active from soma
          nexus_passive_imp.compute(freq_hz + 1 / 9e9, 0) #passive from nexus
          nexus_active_imp.compute(freq_hz + 1 / 9e9, 1) #active from nexus
          for i, seg in enumerate(seg_info):
              elec_dist_info = {
                  'active_soma': soma_active_imp.ratio(seg['x']),
                  'active_nexus': nexus_active_imp.ratio(seg['x']),
                  'passive_soma': soma_passive_imp.ratio(seg['x']),
                  'passive_nexus': nexus_passive_imp.ratio(seg['x'])
              }
              seg_info[i]['seg_elec_info'][freq_name] = elec_dist_info
      return self.__calculate_netcons_per_seg(seg_info)

  def __calculate_netcons_per_seg(self, seg_info):
      NetCon_per_seg = [0] * len(seg_info)
      inh_NetCon_per_seg = [0] * len(seg_info)
      exc_NetCon_per_seg = [0] * len(seg_info)

      v_rest = -60 #used to determine exc/inh may adjust or automate
      
      # calculate number of synapses for each segment (may want to divide by segment length afterward to get synpatic density)
      for netcon in self.netcons_list:
          syn = netcon.syn()
          if syn in self.synapses_list:
              syn_seg_id = seg_info.index(next((s for s in seg_info if s['seg'] == syn.get_segment()), None))
              seg_dict = seg_info[syn_seg_id]
              if syn in seg_dict['seg'].point_processes():
                  NetCon_per_seg[syn_seg_id] += 1 # get synapses per segment
                  if syn.e > v_rest:
                      exc_NetCon_per_seg[syn_seg_id] += 1
                  else:
                      inh_NetCon_per_seg[syn_seg_id] += 1
              else:
                  print("Warning: synapse not in designated segment's point processes")
          else:
              print("Warning: potentially deleted synapse:","|NetCon obj:",netcon,"|Synapse obj:",syn,"the NetCon's synapse is not in synapses_list. Check corresponding original cell's NetCon for location, etc.")
      
      for i, seg in enumerate(seg_info):
          seg['netcons_per_seg'] = {
              'exc': exc_NetCon_per_seg[i],
              'inh': inh_NetCon_per_seg[i],
              'total': NetCon_per_seg[i]
          }
          seg['netcon_density_per_seg'] = {
              'exc': exc_NetCon_per_seg[i]/seg['seg_L'],
              'inh': inh_NetCon_per_seg[i]/seg['seg_L'],
              'total': NetCon_per_seg[i]/seg['seg_L']
          }
      
      return seg_info

In [6]:
%cd expand_example

/content/Model_Reduction_Methods/expand_example


In [7]:
# compile the mod files
!nrnivmodl mod

/content/Model_Reduction_Methods/expand_example
Mod files: "mod/mod/CaDynamics_E2.mod" "mod/mod/Ca_HVA.mod" "mod/mod/Ca_LVAst.mod" "mod/mod/epsp.mod" "mod/mod/Ih.mod" "mod/mod/Im.mod" "mod/mod/K_Pst.mod" "mod/mod/K_Tst.mod" "mod/mod/Nap_Et2.mod" "mod/mod/NaTa_t.mod" "mod/mod/NaTs2_t.mod" "mod/mod/SK_E2.mod" "mod/mod/SKv3_1.mod"

Creating 'x86_64' directory for .o files.

 -> NMODL ../mod/Ca_HVA.mod
 -> NMODL ../mod/Ca_LVAst.mod
 -> Compiling mod_func.cpp
 -> NMODL ../mod/CaDynamics_E2.mod
Translating Ca_HVA.mod into /content/Model_Reduction_Methods/expand_example/x86_64/Ca_HVA.c
Thread Safe
Translating CaDynamics_E2.mod into /content/Model_Reduction_Methods/expand_example/x86_64/CaDynamics_E2.c
 -> NMODL ../mod/epsp.mod
Thread Safe
Translating Ca_LVAst.mod into /content/Model_Reduction_Methods/expand_example/x86_64/Ca_LVAst.c
 -> NMODL ../mod/Ih.mod
Thread Safe
Translating epsp.mod into /content/Model_Reduction_Methods/expand_example/x86_64/epsp.c
Thread Safe
 -> NMODL ../mod/Im.mod
 -

## Setup smiulation parameters

In [8]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import ipywidgets as widgets
from neuron import h
from scipy import signal
from IPython.display import display, clear_output
from ipywidgets import interactive_output, HBox, VBox, Label, Layout

from __future__ import division
from neuron import gui,h
import numpy as np
import time
import matplotlib.pyplot as plt

%matplotlib inline


h.load_file('stdrun.hoc')
# h.nrn_load_dll(paths.COMPILED_LIBRARY_REDUCED_ORDER)  # choose the set of mechanisms
h.nrn_load_dll('./x86_64/.libs/libnrnmech.so')


1.0

### Create a cell with reduced morphology

In [9]:
%ls

cell1.asc  example_expand.py  L5PCtemplate.hoc  x86_64/
Cell.hoc   L5PCbiophys3.hoc   mod/


In [10]:
# syn_to_netcon = {} # dictionary mapping netcons to their synapse
# for netcon in netcons_list: #fill in dictionary
#   syn = netcon.syn() # get the synapse that netcon points to
#   if syn in syn_to_netcon:
#       syn_to_netcon[syn].append(netcon) #add netcon to existing synapse key
#   else:
#       syn_to_netcon[syn] = [netcon] #create new synapse key using netcon as an item

In [11]:
# for netcon in syn_to_netcon[synapses_list[0]]:
#   print(netcon)

In [12]:

h.load_file('L5PCbiophys3.hoc') # load membrane biophysics
h.load_file("import3d.hoc") #load 3d morphology


# Create a cell object
h.load_file('L5PCtemplate.hoc') # load template for generating object
complex_cell = h.L5PCtemplate('cell1.asc') # generate object

#specify some parameters
h.celsius = 37
h.v_init = complex_cell.soma[0].e_pas

#Add synapses to the complex model
synapses_list, netstims_list, netcons_list, randoms_list = [], [], [] ,[]

all_segments = [i for j in map(list,list(complex_cell.apical)) for i in j] + [i for j in map(list,list(complex_cell.basal)) for i in j]
len_per_segment = np.array([seg.sec.L/seg.sec.nseg for seg in all_segments])
rnd = np.random.RandomState(10)
for i in range(10000):
    seg_for_synapse = rnd.choice(all_segments,   p=len_per_segment/sum(len_per_segment)) #choose a random segment with probability based on the length of segment
    synapses_list.append(h.Exp2Syn(seg_for_synapse))
    if rnd.uniform()<0.85: # 85% synapses are excitatory
        e_syn, tau1, tau2, spike_interval, syn_weight = 0, 0.3, 1.8,  1000/2.5, 0.0016
    else: #inhibitory case
        e_syn, tau1, tau2, spike_interval, syn_weight = -86, 1,   8,   1000/15.0, 0.0008
    #set synaptic varibales
    synapses_list[i].e, synapses_list[i].tau1, synapses_list[i].tau2 = e_syn, tau1, tau2
    #set netstim variables
    netstims_list.append(h.NetStim())
    netstims_list[i].interval, netstims_list[i].number, netstims_list[i].start, netstims_list[i].noise = spike_interval, 9e9, 100, 1
    #set random
    randoms_list.append(h.Random())
    randoms_list[i].Random123(i)
    randoms_list[i].negexp(1)
    netstims_list[i].noiseFromRandom(randoms_list[i])       
    #set netcon varibales 
    netcons_list.append(h.NetCon(netstims_list[i], synapses_list[i] ))
    netcons_list[i].delay, netcons_list[i].weight[0] = 0, syn_weight

In [13]:
#reduce each dendritic subtree to a single cable
reduced_cell, synapses_list, netcons_list, txt = subtree_reductor(complex_cell, synapses_list, netcons_list, reduction_frequency=0,return_seg_to_seg=True)

In [14]:
# #insert unused channels for recorder
# for sec in reduced_cell.hoc_model.all:
#   if not hasattr(sec(0.5),'gNaTa_t_NaTa_t'):
#     sec.insert('NaTa_t')
#     for seg in sec:
#       seg.NaTa_t.gNaTa_tbar=0
#     print(sec)

In [15]:
#check synapses_list with netcons_list
for netcon in netcons_list:
  syn=netcon.syn()
  if syn not in synapses_list:
    print(syn, netcon)

In [16]:
#expand cables to idealized dendritic trees
sections_to_expand = [reduced_cell.hoc_model.apic[0]] # expand apical cylinder
furcations_x=[0.289004] #chose using the location of the mapped nexus branching segment
nbranches=[4]
reduced_dendritic_cell, synapses_list, netcons_list, txt = cable_expander(reduced_cell, sections_to_expand, furcations_x, nbranches, 
                                                                          synapses_list, netcons_list, reduction_frequency=0,return_seg_to_seg=True)

2023-04-23 09:16:26 Branching the section using (d)3/2 and electrotonic length rule to preserve service area and electrical properties.
branch_L: 1081.9141048660235 |branch_diam: 1.475351301247016 |trunk_L: 698.0975964944687 |trunk_diam: 3.7176523208618155
2023-04-23 09:16:26 Spreading synapses onto branches
Exp2Syn ['e']
2023-04-23 09:16:26 duplicating branch 1 synapses onto the other branches and randomly distributing Netcons
number of reduced synapses before duplicating synapses to branches: 92
[model[1].apic[1], model[1].apic[2], model[1].apic[3], model[1].apic[4]]
[Exp2Syn[21], Exp2Syn[3]]
Exp2Syn[10000]
Exp2Syn[10001]
Exp2Syn[10002]
consider excluding PP_params: e
Exp2Syn[10003]
consider excluding PP_params: e
Exp2Syn[10004]
consider excluding PP_params: e
Exp2Syn[10005]
[Exp2Syn[42], Exp2Syn[23]]
consider excluding PP_params: e
Exp2Syn[10006]
consider excluding PP_params: e
Exp2Syn[10007]
consider excluding PP_params: e
Exp2Syn[10008]
Exp2Syn[10009]
Exp2Syn[10010]
Exp2Syn[10011]

In [17]:
#check synapses_list with netcons_list
for netcon in netcons_list:
  syn=netcon.syn()
  if syn not in synapses_list:
    print(syn, netcon)

In [18]:
# # try branching the branches - does not work yet
# sections_to_expand = [reduced_dendritic_cell.apic[1],reduced_dendritic_cell.apic[2],reduced_dendritic_cell.apic[3],reduced_dendritic_cell.apic[4]]
# furcations_x=[0.50,0.50,0.50,0.50]
# nbranches=[4,4,4,4]
# reduced_dendritic_cell, synapses_list, netcons_list, txt = cable_expander(reduced_dendritic_cell, sections_to_expand, furcations_x, nbranches, 
#                                                                           synapses_list, netcons_list, reduction_frequency=0,return_seg_to_seg=True)

In [19]:
#check seg mapping
# for i in txt:
#   print(i,"was mapped to",txt[i])

In [20]:
#use defined cell_model python class for generating 3d coordinates, recording ECP, 'book-keeping' etc...
import random
random.seed(2)
cell = cell_model(reduced_dendritic_cell,synapses_list=synapses_list,netcons_list=netcons_list,spike_threshold = 10)
# cell._nbranch=4

Warning message comes from random chance that synapse did not recieve a netcon after redistributing from single cable to equivalent branches.

Regenerate Original Cell

In [21]:
orig_cell = h.L5PCtemplate('cell1.asc') # generate object

#Add synapses to the complex model
synapses_list, netstims_list, netcons_list, randoms_list = [], [], [] ,[]

all_segments = [i for j in map(list,list(orig_cell.apical)) for i in j] + [i for j in map(list,list(orig_cell.basal)) for i in j]
len_per_segment = np.array([seg.sec.L/seg.sec.nseg for seg in all_segments])
rnd = np.random.RandomState(10)
for i in range(10000):
    seg_for_synapse = rnd.choice(all_segments,   p=len_per_segment/sum(len_per_segment)) #choose a random segment with probability based on the length of segment
    synapses_list.append(h.Exp2Syn(seg_for_synapse))
    if rnd.uniform()<0.85: # 85% synapses are excitatory
        e_syn, tau1, tau2, spike_interval, syn_weight = 0, 0.3, 1.8,  1000/2.5, 0.0016
    else: #inhibitory case
        e_syn, tau1, tau2, spike_interval, syn_weight = -86, 1,   8,   1000/15.0, 0.0008
    #set synaptic varibales
    synapses_list[i].e, synapses_list[i].tau1, synapses_list[i].tau2 = e_syn, tau1, tau2
    #set netstim variables
    netstims_list.append(h.NetStim())
    netstims_list[i].interval, netstims_list[i].number, netstims_list[i].start, netstims_list[i].noise = spike_interval, 9e9, 100, 1
    #set random
    randoms_list.append(h.Random())
    randoms_list[i].Random123(i)
    randoms_list[i].negexp(1)
    netstims_list[i].noiseFromRandom(randoms_list[i])       
    #set netcon varibales 
    netcons_list.append(h.NetCon(netstims_list[i], synapses_list[i] ))
    netcons_list[i].delay, netcons_list[i].weight[0] = 0, syn_weight

In [22]:
original_cell = cell_model(model=orig_cell,
                           synapses_list=synapses_list,netcons_list=netcons_list,
                           gen_3d=False,gen_geom_csv=False,
                           spike_threshold = 10)

In [23]:
#may be able to delete
def get_syn_to_netcons(netcons_list):
    syn_to_netcon = {} # dictionary mapping netcons to their synapse
    for netcon in netcons_list: #fill in dictionary
      syn = netcon.syn() # get the synapse that netcon points to
      if syn in syn_to_netcon:
          syn_to_netcon[syn].append(netcon) #add netcon to existing synapse key
      else:
          syn_to_netcon[syn] = [netcon] #create new synapse key using netcon as an item
    return syn_to_netcon

syn_to_netcon=get_syn_to_netcons(netcons_list)

In [24]:
# #get expanded segments dataframe
# make_reduced_seg_df(cell,"segments_expanded.csv") #need to improve make_reduced_seg_df
# expanded_segments_df=pd.read_csv("segments_expanded.csv")

# #get reduced segments dataframe
# make_reduced_seg_df(original_cell,"segments_original.csv") #need to improve make_reduced_seg_df
# original_segments_df=pd.read_csv(original_cell"segments_original.csv")

# # change to complex cell
# # make_reduced_seg_df(cell,"segments_expanded.csv") #need to improve make_reduced_seg_df
# # expanded_segments_df=pd.read_csv("segments_expanded.csv")
# # plot_morphology(expanded_segments_df,"expanded_morphology.svg")

In [25]:
# dir(cell)

Mount ECP dir may not be necessary

In [26]:
# #mount ECP dir # may only need this for ECP notebook
# import os

# RunningInCOLAB = 'google.colab' in str(get_ipython())
# if RunningInCOLAB:
#     !pip install neuron==8.0.0 &> /dev/null
#     os.chdir('/content')
#     if not os.path.isdir('Stylized-Single-Cell-and-Extracellular-Potential'):
#         !git clone https://github.com/chenziao/Stylized-Single-Cell-and-Extracellular-Potential.git &> /dev/null 
#     os.chdir('Stylized-Single-Cell-and-Extracellular-Potential')
#     %ls

In [27]:
dir(original_cell)

['Vm',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get_segment_info__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cell_model__calc_seg_coords',
 '_cell_model__calculate_netcons_per_seg',
 '_cell_model__convert_sectionlist',
 '_cell_model__create_output_files',
 '_cell_model__create_output_folder',
 '_cell_model__generate_geometry_file',
 '_cell_model__generate_sec_coords',
 '_cell_model__get_parent_segment_ids',
 '_cell_model__get_segment_elec_dist',
 '_cell_model__insert_unused_channels',
 '_cell_model__report_data',
 '_cell_model__set_spike_recorder',
 '_cell_model__setup_recorders',
 '_cell_model__store_point_processes',
 '_cell_model__store_segments',
 '_cell_model__store_synapses_list',
 '_nseg'

In [28]:
# original_cell.write_seg_info_to_csv()
# original_segments_df=pd.read_csv('/'+original_cell.output_folder_name+'/seg_info.csv')

In [29]:
# cell.__write_seg_info_to_csv()
# expanded_segments_df=pd.read_csv('/'+cell.output_folder_name+'/seg_info.csv')

In [30]:
def run_simulation(tstop=5000,dt=0.1):
    '''
    Run simulation and print the runtime
    tstop :  ms simulation duration
    '''
    # Define simulation parameters
    h.tstop = tstop
    h.dt = dt
    h.steps_per_ms = 1/h.dt

    # Define simulation start time
    timestart = time.time()

    # Run simulation
    h.run()

    # Define simulation stop time
    timestop = time.time()

    # Calculate and print the runtime
    elapsedtime = timestop - timestart
    simtime = tstop/1000  # convert from ms to s
    print(f"It took {round(elapsedtime)} sec to run a {simtime} sec simulation.")

    return

In [31]:
run_simulation(tstop=5000)
# #Run simulation

# timestart=time.time()
# h.run()
# timestop=time.time()
# t = h.t # was t=h.t() but 'float' object not callable
# elapsedtime=timestop-timestart
# simtime=tstop/1000 #convert from ms to s
# # totaltime= elapsedtime+elapseddeftime
# print('It took',round(elapsedtime),'sec to run a',simtime,'sec simulation.')
# # print('The total runtime was',round(totaltime),'sec')

It took 209 sec to run a 5.0 sec simulation.


In [32]:
# import h5py
# def createsegtracereport(reportname,dataname):
#   try:
#     os.remove(reportname) # reportname was string " "
#   except:
#     x = 1

#   f = h5py.File(reportname,'w') #create a file in the w (write) mode #reportname was string ' '
#   v = f.create_dataset("report/biophysical/data", data = dataname)
#   f.close()

In [33]:
# def calculate_net_synaptic_currents(cell):
#     numTstep = int(h.tstop/h.dt)
#     i_NMDA_bySeg = [[0] * (numTstep+1)] * len(cell.segments)
#     i_AMPA_bySeg = [[0] * (numTstep+1)] * len(cell.segments)
#     i_bySeg = [[0] * (numTstep+1)] * len(cell.segments)

#     for synapse in cell.synapses_list:
#         try:
#             i_NMDA = np.array(synapse.rec_vec.vec_list[1])
#             i_AMPA = np.array(synapse.rec_vec.vec_list[0])
#             seg = synapse.get_segment_id()

#             try:
#                 i_NMDA_bySeg[seg] = i_NMDA_bySeg[seg] + i_NMDA
#                 i_AMPA_bySeg[seg] = i_AMPA_bySeg[seg] + i_AMPA
#                 i_bySeg[seg] = i_bySeg[seg] + i_NMDA + i_AMPA
#             except:
#                 pass
#         except:
#             pass
    
#     return i_NMDA_bySeg, i_AMPA_bySeg, i_bySeg

In [34]:
#calculate inmda
#since synapse list has combined synapses for computational efficiency, we must use NetCon list/unique spike trains to determine how many synapses were mapped

# v_rest=-60 #choose v_rest for categorizing inh/exc synapses

# NetCon_per_seg=[0]*len(cell.segments)
# inh_NetCon_per_seg=[0]*len(cell.segments)
# exc_NetCon_per_seg=[0]*len(cell.segments)

# #calculate number of synapses for each segment (may want to divide by segment length afterward to get synpatic density)
# for netcon in cell.netcons_list:
#   syn=netcon.syn()
#   if syn in synapses_list:
#     syn_seg_id=cell.segments.index(netcon.syn().get_segment())
#     if syn in cell.segments[syn_seg_id].point_processes():
#       NetCon_per_seg[syn_seg_id]+=1 # get synapses per segment
#       # NetCon_per_seg[syn_seg_id].append(netcon) # possible implementation if needing objects per segment
#       if syn.e > v_rest:
#         exc_NetCon_per_seg[syn_seg_id]+=1
#         # exc_NetCon_per_seg[syn_seg_id].append(netcon)# possible implementation if needing objects per segment
#       else:
#         inh_NetCon_per_seg[syn_seg_id]+=1
#         # inh_NetCon_per_seg[syn_seg_id].append(netcon)# possible implementation if needing objects per segment
#     else:
#       print("Warning: synapse not in designated segment's point processes")

#   else:
#     print("Warning: potentially deleted synapse:","|NetCon obj:",netcon,"|Synapse obj:",syn,"the NetCon's synapse is not in synapses_list. Check corresponding original cell's NetCon for location, etc.")

# numTstep = int(h.tstop/h.dt)
# i_NMDA_bySeg= [[0] * (numTstep+1) ] * len(cell.segments) # need to implement inmda recording

# #extract inmda from each segment # can be adjusted for gaba synapses or alpha synapses # only paired nmda/ampa synapses have vec_list[1]
# for synapse in cell.synapse:
#   try:
#     i_NMDA = np.array(synapse.rec_vec.vec_list[1])            #current = numpy array of NEURON Vector of current NMDA current at synapse j  
#     seg = synapse.get_segment_id()                            #seg = the segment in which synapse j is located 
#     #print('first try')
    
#     try:
#       i_NMDA_bySeg[seg] = i_NMDA_bySeg[seg] + i_NMDA    
#       #print('second try')                                           #Sum current over each segment
#     except: 
#       pass                                                                   #Except needed as some synpases do not have NMDA currrent and throw an error when called
#   except:
#     pass

In [35]:
# def calculate_netcons_per_seg(self):
#   NetCon_per_seg=[0]*len(self.segments)
#   inh_NetCon_per_seg=[0]*len(self.segments)
#   exc_NetCon_per_seg=[0]*len(self.segments)

#   #calculate number of synapses for each segment (may want to divide by segment length afterward to get synpatic density)
#   for netcon in self.netcons_list:
#     syn=netcon.syn()
#     if syn in synapses_list:
#       syn_seg_id=self.segments.index(netcon.syn().get_segment())
#       if syn in self.segments[syn_seg_id].point_processes():
#         NetCon_per_seg[syn_seg_id]+=1 # get synapses per segment
#         # NetCon_per_seg[syn_seg_id].append(netcon) # possible implementation if needing objects per segment
#         if syn.e > v_rest:
#           exc_NetCon_per_seg[syn_seg_id]+=1
#           # exc_NetCon_per_seg[syn_seg_id].append(netcon)# possible implementation if needing objects per segment
#         else:
#           inh_NetCon_per_seg[syn_seg_id]+=1
#           # inh_NetCon_per_seg[syn_seg_id].append(netcon)# possible implementation if needing objects per segment
#       else:
#         print("Warning: synapse not in designated segment's point processes")

#     else:
#       print("Warning: potentially deleted synapse:","|NetCon obj:",netcon,"|Synapse obj:",syn,"the NetCon's synapse is not in synapses_list. Check corresponding original cell's NetCon for location, etc.")
#   return NetCon_per_seg,inh_NetCon_per_seg,exc_NetCon_per_seg

In [36]:
# expanded_segments_df["exc_NetCons"]=cell.exc_NetCon_per_seg
# expanded_segments_df["inh_NetCons"]=cell.inh_NetCon_per_seg
# expanded_segments_df["inh_NetCon_density"]=cell.expanded_segments_df["inh_NetCons"]/expanded_segments_df["Seg_L"]
# expanded_segments_df["exc_NetCon_density"]=cell.expanded_segments_df["exc_NetCons"]/expanded_segments_df["Seg_L"]
# expanded_segments_df.to_csv("segments_expanded.csv") #update saved data to include above columns. 
# #Can instead include above columns and calculation in "make_reduced_seg_df()"

In [37]:
# #subset dataframe to ignore certain segments and see synaptic density of branches
# ignore=[]
# for seg in cell.apic[0]:#ignore trunk
#   ind=cell.segments.index(seg)
#   ignore.append(ind)
# df=expanded_segments_df[~expanded_segments_df.index.isin(ignore)]

In [38]:
# #can stow this function away
# def plot_seg_heatmap(seg_df,label=str,savename=str,color_column=str):
#   '''color_column: name of the column that you want to use as the parameter for heatmap color'''
#   plt.figure(figsize=(4,10))
#   ax = plt.scatter(seg_df["Coord X"], seg_df["Coord Y"],c = seg_df[color_column],cmap='jet',)
#   plt.vlines(110,400,500)
#   plt.text(0,450,'100 um')
#   plt.hlines(400,110,210)
#   plt.text(110,350,'100 um')
#   plt.xticks([])
#   plt.yticks([])
#   cbar = plt.colorbar()
#   cbar.ax.set_ylabel(label, rotation=270)

#   plt.box(False)
#   plt.savefig(savename+'.svg')

In [39]:
# output_folder_name=(
#     "outputcontrol_"+
#     str(nbranches[0])+"nbranch_"+
#     str(int(len(cell.netcons_list)))+"NCs_"
#     +str(int(len(cell.synapses_list)))+"nsyn_"+
#     str(int(len(cell.segments)))+"nseg"
#                                         )#+modelname  #include model name in output foler name (ex. original cell, reduced cell, expanded cell)
# #create output folder
# import os
# if not os.path.exists(output_folder_name):
#    os.makedirs(output_folder_name)

# print(output_folder_name)
# # os.chdir(output_folder_name)

In [40]:
# def create_output_folder(cell):
#     nbranches = len(cell.apic)-1
#     nc_count = len(cell.netcons_list)
#     syn_count = len(cell.synapses_list)
#     seg_count = len(cell.segments)
    

#     output_folder_name = (
#         str(h.tstop)+
#         "outputcontrol_" +
#         str(nbranches) + "nbranch_" +
#         str(nc_count) + "NCs_" +
#         str(syn_count) + "nsyn_" +
#         str(seg_count) + "nseg"
#     )

#     if not os.path.exists(output_folder_name):
#         os.makedirs(output_folder_name)

#     return output_folder_name

In [55]:
def plot_morphology(seg_df, savename):
    plt.figure(figsize=(4, 10))
    for i in seg_df[seg_df.type == 'apic']['Sec ID'].unique():
        plt.plot(
            seg_df[(seg_df.type == 'apic') & (seg_df['Sec ID'] == i)]['Coord X'],
            seg_df[(seg_df.type == 'apic') & (seg_df['Sec ID'] == i)]['Coord Y'],
            color='k',
            linewidth=2 * seg_df[(seg_df.type == 'apic') & (seg_df['Sec ID'] == i)]['Section_diam'].unique()
        )
    for i in seg_df[seg_df.type == 'dend']['Sec ID'].unique():
        plt.plot(
            seg_df[(seg_df.type == 'dend') & (seg_df['Sec ID'] == i)]['Coord X'],
            seg_df[(seg_df.type == 'dend') & (seg_df['Sec ID'] == i)]['Coord Y'],
            color='k',
            linewidth=2 * seg_df[(seg_df.type == 'dend') & (seg_df['Sec ID'] == i)]['Section_diam'].unique()
        )

    # save the figure
    plt.axis('off')
    plt.savefig(savename, bbox_inches='tight')
    plt.close()

In [42]:
original_cell.get_recorder_data() #also creates directory

Outputting data to  5000.0outputcontrol_108nbranch_10000NCs_10000nsyn_642nseg


{'spikes': Vector[5455],
 'ih_data': array([[-1.11185907e-03, -1.11185907e-03, -1.10991652e-03, ...,
         -3.18233396e-05, -3.17763926e-05, -3.17166831e-05],
        [-1.11185907e-03, -1.11185907e-03, -1.11010559e-03, ...,
         -3.25805061e-05, -3.25803868e-05, -3.25761445e-05],
        [-1.11185907e-03, -1.11185907e-03, -1.11027407e-03, ...,
         -3.35719760e-05, -3.36590309e-05, -3.37550970e-05],
        ...,
        [-2.28742766e-03, -2.28742766e-03, -2.28187396e-03, ...,
         -6.81899692e-05, -6.80538974e-05, -6.79403582e-05],
        [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00, ...,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
        [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00, ...,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00]]),
 'gNaTa_T_data': array([[3.23378406e-11, 3.23378406e-11, 3.34312556e-11, ...,
         6.89000734e-06, 7.00333674e-06, 7.13415041e-06],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 

In [ ]:
original_cell.write_seg_info_to_csv()

In [46]:
original_segments_df=pd.read_csv(original_cell.output_folder_name+'/seg_info.csv')

In [49]:
print(original_segments_df.columns)

Index(['seg', 'seg_id', 'bmtk_id', 'x', 'type', 'sec_id', 'nseg', 'Ra',
       'seg_L', 'parent_seg_id', 'seg_elec_info', 'netcons_per_seg',
       'netcon_density_per_seg'],
      dtype='object')


In [54]:
plot_morphology(original_segments_df,"/"+original_cell.output_folder_name+"morphology.svg")

KeyError: ignored

<Figure size 400x1000 with 0 Axes>

In [ ]:
cell.get_recorder_data() #also creates directory

In [ ]:
cell.__write_seg_info_to_csv() 
expanded_segments_df=pd.read_csv('/'+cell.output_folder_name+'/seg_info.csv')
plot_morphology(expanded_segments_df,"/"+cell.output_folder_name+"morphology.svg")

Excitatory synapse density

In [ ]:
original_cell.plot_seg_heatmap(original_segments_df,color_column="exc_NetCon_density")

In [ ]:
cell.plot_seg_heatmap(expanded_segments_df,color_column="exc_NetCon_density")

Inhibitory synapse density

In [ ]:
original_cell.plot_seg_heatmap(original_segments_df,color_column="inh_NetCon_density")

In [ ]:
cell.plot_seg_heatmap(expanded_segments_df,color_column="inh_NetCon_density")

In [ ]:
# import h5py
# def createsegtracereport(reportname,dataname):
#   try:
#     os.remove(reportname) # reportname was string " "
#   except:
#     x = 1

#   f = h5py.File(reportname,'w') #create a file in the w (write) mode #reportname was string ' '
#   v = f.create_dataset("report/biophysical/data", data = dataname)
#   f.close()

# # output files for analysis in another notebook
# createsegtracereport(output_folder_name+'/v_report.h5', Vm.T)
# createsegtracereport(output_folder_name+'/Ca_HVA.ica_report.h5',icah_data.T)
# createsegtracereport(output_folder_name+'/Ca_LVAst.ica_report.h5',ical_data.T)
# createsegtracereport(output_folder_name+'/Ih.ihcn_report.h5',ih_data.T)
# createsegtracereport(output_folder_name+'/inmda_report.h5',i_NMDA_df.T)
# createsegtracereport(output_folder_name+'/NaTa_t.gNaTa_t_report.h5',gNaTa_T_data.T)
# createsegtracereport(output_folder_name+'/NaTa_t.ina_report.h5',ina_data.T)

In [ ]:
# try:
#   os.remove(output_folder_name+"/spikes.h5")
# except:
#   x = 1

# f = h5py.File(output_folder_name+'/spikes.h5','w') #create a file in the w (write) mode
# v = f.create_dataset("spikes/biophysical/timestamps", data = cell.spikes)

# f.close()

In [ ]:
#adjust so that a folder is generated for complex and reduced cell and with simulation time as part of the folder name
#maybe include syn distribution in name too

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !zip -r <"outputcontrol_4nbranch_10000NCs_114nsyn_88nseg">.zip <"/content/Stylized-Single-Cell-and-Extracellular-Potential/outputcontrol_4nbranch_10000NCs_114nsyn_88nseg">

In [ ]:
# output_folder_zip=output_folder_name+'.zip'
# # 

In [ ]:
# !zip -r $output_folder_zip $output_folder_name

In [ ]:
# /content/Stylized-Single-Cell-and-Extracellular-Potential/outputcontrol_4nbranch_10000NCs_114nsyn_88nseg

In [ ]:
# # expanded_segments_df.to_csv('expanded_segments_df.csv')
# # from google.colab import files
# # Save the DataFrame to the specified file path
# expanded_segments_df.to_csv('/content/drive/MyDrive/expanded_segments_df.csv', index=False)

In [ ]:
# %ls

In [ ]:
# !cp $output_folder_zip /content/drive/MyDrive/

In [ ]:
# %cd outputcontrol_4nbranch_10000NCs_114nsyn_88nseg/